In [ ]:
%load_ext autoreload  
%autoreload 2
# so you dont have to restart the Kernel whenever utils updated

#TODO: Fix move logic - move partnering logic out of base class, repetition in age logic - ask jonny if ok, beavers dont pair up again!, 


""" Agent """

from mesa.experimental.cell_space import CellAgent

class Beaver(CellAgent):
    """Base Beaver Class"""

    def __init__(self, model, sex=None, cell=None, age=0):
        """
		* Initialise and populate the model
		"""
        super().__init__(model) 
        self.sex = sex if sex else model.random.choice(['M', 'F'])
        self.cell = cell
        self.partner = None
        self.age = age
        self.reproduction_timer = 0

    def step(self):
        if self.partner is None:
            potential_mates = [
                a for a in self.cell.agents
                if isinstance(a, Beaver) and a.sex != self.sex and a.partner is None
            ]
            if potential_mates:
                mate = self.random.choice(potential_mates)
                self.partner = mate
                mate.partner = self

        # move together if paired, else move alone
        if self.partner and self.partner.partner == self:
            if self.unique_id < self.partner.unique_id:  # only one of the pair moves both
                self.move(together=True)
        else:
            self.move(together=False)

    def move(self, together=False):
        new_cell = self.cell.neighborhood.select_random_cell()
        self.move_to(new_cell)
        if together and self.partner:
            self.partner.move_to(new_cell)
       

    def reproduce(self):
        if self.partner and self.cell is not None:
            for _ in range(self.random.randint(1, 3)): # random number of kits between 1-3
                kit = Kit(self.model, cell=self.cell)
                self.cell.agents.append(kit)
                self.model._agents_by_type[Beaver].append(kit)

    def age_up(self):
        # kit -> juvenile at age 2 (24 steps), juvenile -> adult at age 3 (36 steps)
        if isinstance(self, Kit) and self.age >= 24: 
            return Juvenile(self.model, sex=self.sex, cell=self.cell, age=self.age)
        elif isinstance(self, Juvenile) and self.age >= 36:
            return Adult(self.model, sex=self.sex, cell=self.cell, age=self.age)
        else:
            return self


class Kit(Beaver):
    # kits move with group, can't pair or reproduce, age up
    def step(self): 
        self.move() # specific movement logic - move with colony
        self.age += 1  

        new_self = self.age_up() # age up if applicable
        if new_self is not self:
            self.cell.agents.remove(self)
            self.model._agents_by_type[Beaver].remove(self)
            self.cell.agents.append(new_self)
            self.model._agents_by_type[Beaver].append(new_self)
            return new_self.step()


class Juvenile(Beaver):
    # juveniles disperse away from group, pair and reproduce, !build dams!, age up
    def step(self):
        self.move()
        self.age += 1  

        # reproduction logic 
        if self.partner and self.partner.partner == self and self.unique_id < self.partner.unique_id:
            self.reproduction_timer += 1
            if self.reproduction_timer >= 12:
                self.reproduce()
                self.reproduction_timer = 0
        else:
            self.reproduction_timer = 0

        
        new_self = self.age_up() # age up if applicable
        if new_self is not self:
            self.cell.agents.remove(self)
            self.model._agents_by_type[Beaver].remove(self)
            self.cell.agents.append(new_self)
            self.model._agents_by_type[Beaver].append(new_self)
            return new_self.step()


class Adult(Beaver):
    # adults have full range of beaver behaviour (pairing, moving, reproducing, !building dams!, they dont age up-they die)
    def step(self):
        self.age += 1
        super().step()  # call base beaver logic (pairing, movement)

        # reproduction logic 
        if self.partner and self.partner.partner == self and self.unique_id < self.partner.unique_id:
            self.reproduction_timer += 1
            if self.reproduction_timer >= 12:
                self.reproduce()
                self.reproduction_timer = 0
        else:
            self.reproduction_timer = 0

        #TODO: partners dont repair when partner dies - they also dont move! fix
        if self.age >= 84: 
            #if self.partner:
                #self.partner.partner = None
            #if self in self.cell.agents: #was trying to remove partner too - checks agent is in list before removing them from it? - still results in error
                self.cell.agents.remove(self)
            #if self in self.model._agents_by_type[Beaver]:
                self.model._agents_by_type[Beaver].remove(self)
                return





""" Model """

from mesa import Model
from mesa.datacollection import DataCollector
from mesa.experimental.cell_space import OrthogonalVonNeumannGrid
from mesa.experimental.devs import ABMSimulator

#from beaver_agent import Beaver  # if this is seperate files


class BeaverModel(Model):
    def __init__(self, width=20, height=20, initial_beavers=50, seed=None, simulator=None): # initialise
        super().__init__(seed=seed)
        self.width = width
        self.height = height

        # properly initialise the grid
        self.grid = OrthogonalVonNeumannGrid(
            [self.height, self.width],
            torus=True,
            capacity=float("inf"),
            random=self.random,
        )

        # initialise agents_by_type as a set NOT list
        self._agents_by_type = {Beaver: []}

        # create initial beavers and add them to the grid
        for _ in range(initial_beavers):
            cell = self.random.choice(self.grid.all_cells.cells)
            beaver = Adult(model=self, cell=cell) # add only adult beavers 
            cell.agents.append(beaver)
            self._agents_by_type[Beaver].append(beaver)


        self.datacollector = DataCollector({
            "Beavers": lambda m: len(m._agents_by_type[Beaver]),
            "Paired Beavers": lambda m: len(
                [a for a in m._agents_by_type[Beaver] if a.partner and a.unique_id < a.partner.unique_id]
            ),
            "Males": lambda m: len([a for a in m._agents_by_type[Beaver] if a.sex == "M"]),
            "Females": lambda m: len([a for a in m._agents_by_type[Beaver] if a.sex == "F"]),
            "Kits": lambda m: len([a for a in m._agents_by_type[Beaver] if isinstance(a, Kit)]),
            "Juveniles": lambda m: len([a for a in m._agents_by_type[Beaver] if isinstance(a, Juvenile)]),
            "Adults": lambda m: len([a for a in m._agents_by_type[Beaver] if isinstance(a, Adult)]),
        })
        self.datacollector.collect(self)

        if simulator is not None:
            self.simulator = simulator
            self.simulator.setup(self)
            
        self.running = True

    def step(self):
        # update the agents
        for agent in list(self._agents_by_type[Beaver]):
          agent.step()
        
        self.datacollector.collect(self) # collect data on each step



""" App """

from mesa.experimental.devs import ABMSimulator
from mesa.visualization import (
    Slider,
    SolaraViz,
    make_plot_component,
    make_space_component,
)

#from beaver_model import BeaverModel  # your adapted model
#from beaver_agent import Beaver  # your Beaver agent class


def beaver_portrayal(agent):
    if not getattr(agent, "cell", None):
        return None  # skip agents with no cell

    portrayal = {
        "size": 25,
        "marker": "o",
        "zorder": 2,
    }

    #TODO: 
    if isinstance(agent, Beaver):
        if agent.partner is not None:
            portrayal["color"] = "purple"
        elif agent.sex == "M":
            portrayal["color"] = "blue"
        else:
            portrayal["color"] = "red"

    if isinstance(agent, Kit):
        portrayal["color"] = "green"
    elif isinstance(agent, Juvenile):
        portrayal["color"] = "orange"
    elif isinstance(agent, Adult):
        portrayal["color"] = "brown"
    else:
        portrayal["color"] = "gray"

    return portrayal



model_params = {
    "seed": {"type": "InputText", "value": 42,"label": "Random Seed" },
    "initial_beavers": Slider("Initial Beaver Population", 50, 10, 200),
    "width": Slider("Grid Width", 20, 5, 50),
    "height": Slider("Grid Height", 20, 5, 50),
}

def post_process_space(ax):
    ax.set_aspect("equal")
    ax.set_xticks([])
    ax.set_yticks([])

def post_process_lines(ax):
    ax.legend(loc="center left", bbox_to_anchor=(1, 0.9))

space_component = make_space_component(
    beaver_portrayal, draw_grid=False, post_process=post_process_space
)

lineplot_component = make_plot_component(
    {
        "Beavers": "tab:gray",
        "Males": "blue",
        "Females": "red",
        "Paired Beavers": "purple",
        "Kits": "green",
        "Juveniles": "orange",
        "Adults": "brown",
    },
    post_process=post_process_lines,
)

simulator = ABMSimulator()
model = BeaverModel(simulator=simulator)

page = SolaraViz(
    model,  
    components=[space_component, lineplot_component],
    model_params=model_params,
    name="Beaver Simulation",
    simulator=simulator,
)

page  # noqa

/opt/miniconda3/envs/abm/lib/python3.13/site-packages/mesa/experimental/cell_space/property_layer.py:226: FutureWarning: The property layer functionality and associated classes are experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1932
  layer = PropertyLayer(


Cannot show ipywidgets in text